# Sparse Identification of Nonlinear Dynamics (SINDy)

The goal is to recreate the methods and results of the chaotic Lorenz system example. First we will generate the data from the dynamical equations below, then we will use the sparse regression method LASSO to recover the equations from said data.

## Generating Lorenz data

The Lorenz system is governed by the equations 
$$\dot{x}=\sigma(y-x)$$
$$\dot{y}=x(\rho - z)-y$$
$$\dot{z}=xy-\beta z$$


In [56]:
function xdot(x,y,sigma)
    xdot = sigma*(y-x)
    return xdot
end
function ydot(x,y,z,rho)
    ydot = x*(rho-z)-y
    return ydot
end
function zdot(x,y,z,beta)
    zdot = x*y-beta*z
    return zdot
end

zdot (generic function with 1 method)

Lets evolve these equations using the RK4 method. 

Initial conditions used in Brunton paper are $(x_0, y_0, z_0) = (-8,7,27)$ and constants $\sigma =10,\beta=8/3,\rho=28$.

In [57]:
x0 = -8;y0 = 7;z0 = 27;                     # Initial Condition

In [58]:
sigma=10;beta=8/3;rho=28;

In [6]:
using PyCall
np = pyimport("numpy")

PyObject <module 'numpy' from 'C:\\Users\\shane\\.julia\\conda\\3\\lib\\site-packages\\numpy\\__init__.py'>

In [73]:
h=0.001;                                    # Time step
t = np.linspace(0,100,floor(Int,100/h) )               

x = zeros(length(t));  
y = zeros(length(t));  
z = zeros(length(t));  
xd = zeros(length(t));  
yd = zeros(length(t));  
zd = zeros(length(t));  

x[1] = x0;
y[1] = y0;
z[1] = z0;
xd[1] = xdot(x[1],y[1],sigma);
yd[1] = ydot(x[1],y[1],z[1],rho);
zd[1] = zdot(x[1],y[1],z[1],beta);

for i in 1:(length(t)-1)
  
    k1x = xdot(x[i],y[i],sigma)                   # Approx for x gives approx for deriv
    k1y = ydot(x[i],y[i],z[i],rho) 
    k1z = zdot(x[i],y[i],z[i],beta)
    x1 = x[i]+k1x*h/2;                                 # Intermediate value (using k1)
    y1 = y[i]+k1y*h/2;      
    z1 = z[i]+k1z*h/2;      

    k2x = xdot(x1,y1,sigma)        # Approx deriv at intermediate value.
    k2y = ydot(x1,y1,z1,rho)
    k2z = zdot(x1,y1,z1,beta)
    x2 = x[i]+k2x*h/2;            # Intermediate value (using k2)
    y2 = y[i]+k2y*h/2;
    z2 = z[i]+k2z*h/2;
  
    k3x = xdot(x2,y2,sigma)        # Approx deriv at intermediate value.
    k3y = ydot(x2,y2,z2,rho)
    k3z = zdot(x2,y2,z2,beta)
    x3 = x[i]+k3x*h/2;            # Intermediate value (using k2)
    y3 = y[i]+k3y*h/2;
    z3 = z[i]+k3z*h/2;
    
    k4x = xdot(x3,y3,sigma)        # Approx deriv at endpoint value.
    k4y = ydot(x3,y3,z3,rho)
    k4z = zdot(x3,y3,z3,beta)
    
    x[i+1] = x[i] + (k1x+2*k2x+2*k3x+k4x)*h/6; # Approx soln
    y[i+1] = y[i] + (k1y+2*k2y+2*k3y+k4y)*h/6;
    z[i+1] = z[i] + (k1z+2*k2z+2*k3z+k4z)*h/6;
    xd[i+1] = k4x;
    yd[i+1] = k4y;
    zd[i+1] = k4z;
    
end

In [19]:
using Plots

In [77]:
l = @layout [a  ; b]

p1 = plot(x, y, z, size=(800,800))

p2 = plot(xd,yd,zd, size=(800,800))
plot(p1, p2, layout = l)

Lets collect the data into 2 matrices $\textbf{X}$ and $\dot{\textbf{X}}$ as in Brunton.

In [75]:
X = (x,y,z);
Xd = (xd,yd,zd);

For the Lorenz model we choose a basis of polynomial nonlinearities up to 5th order. The relevant functions are

- '$1$'
- '$x, y, z$'
- '$x^2, xy, ... , zy, z^2$'
- '$x^3, x^2y, ... , z^2y, z^3$'
- '$x^4, x^3y, ... , z^3y, z^4$'
- '$x^5, x^4y, ... , z^4y, z^5$'